<a href="https://colab.research.google.com/github/as9786/ParrotnlpJeju/blob/main/0402%EB%B0%9C%ED%91%9C1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, Input, GRU, Bidirectional, GlobalMaxPooling1D, Dropout, Flatten,SpatialDropout1D,Conv1D,concatenate
from tensorflow.keras.layers import GlobalAveragePooling1D, add, ReLU
#Bidirectional은 양방향으로 데이터를 보기 위해서 사용되는 함수
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')

In [ ]:
X_train = train_df.comment_text
y_train = train_df[["toxic",	"severe_toxic",	"obscene",	"threat",	"insult",	"identity_hate"]].values

In [ ]:
num_words = 70000
max_len = 150

In [ ]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(list(X_train))
word_index = tokenizer.word_index

In [ ]:
# tokenizer의 num_words가 단어의 개수를 제어해주지 못하기 때문에 인위적으로 단어 개수 제한
word_index = {e:i for e,i in word_index.items() if i <= num_words}

In [ ]:
len(word_index)

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)

In [ ]:
X_train = sequence.pad_sequences(X_train, maxlen = max_len)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

In [ ]:
n=0
f = open('glove.6B.300d.txt', encoding="utf8") # 단어를 300차원으로 표현

for line in f:
    word_vector = line.split() # 각 줄을 읽어와서 word_vector에 저장.
    print(word_vector) # 각 줄을 출력
    word = word_vector[0] # word_vector에서 첫번째 값만 저장
    print(word) # word_vector의 첫번째 값만 출력
    n=n+1
    if n==2:
        break
f.close()

In [ ]:

print(type(word_vector))
print(len(word_vector))

In [ ]:
import numpy as np
embedding_dict = dict()
f = open('glove.6B.300d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 300개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()
print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

In [ ]:
print(embedding_dict['respectable'])
print(len(embedding_dict['respectable']))

In [ ]:
embedding_matrix = np.zeros((num_words, 300))
# 단어 집합 크기의 행과 300개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

In [ ]:
embedding_matrix[:10]

In [ ]:
for word, i in word_index.items():
  tmp = embedding_dict.get(word)
  if tmp is not None:
    embedding_matrix[i-1] = tmp

In [ ]:
input = Input(shape=(max_len, )) 
layer = Embedding(num_words, 300, weights=[embedding_matrix], input_length=max_len, trainable=True)(input)
layer = SpatialDropout1D(0.2)(layer)
layer = Bidirectional(GRU(128, return_sequences=True))(layer)
layer = Conv1D(128, kernel_size = 3)(layer)   
hidden = concatenate([GlobalMaxPooling1D()(layer),GlobalAveragePooling1D()(layer)])
hidden = Dropout(0.25)(hidden)
result= Dense(6, activation="sigmoid")(hidden)
model = Model(inputs = input, outputs = result)  
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, y_train, batch_size=512, epochs=4, validation_split=0.2)

In [ ]:
model.save('/content/drive/MyDrive/toxic7_model0401.h5')

In [ ]:
import matplotlib.pyplot as plt
def acc_loss_plot(hist):
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc = 'upper left')

    acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc='upper right')

    plt.show()

In [ ]:
acc_loss_plot(hist)

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')
test_df = test_df['comment_text']
test_final = tokenizer.texts_to_sequences(test_df)
test_padded =sequence.pad_sequences(test_final, maxlen=150)

In [ ]:
predict = model.predict(test_padded)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = predict
submission.to_csv('/content/drive/MyDrive/submission7.csv', index=False)